In [6]:
import argparse

import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from tqdm import tqdm

try:
    from apex import amp

except ImportError:
    amp = None

# from dataset import LMDBDataset
from pixelsnail import PixelSNAIL
from scheduler import CycleScheduler

import argparse
import sys
import os
from skimage import io

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, utils
import matplotlib.patches as mpatches


# from scheduler import CycleScheduler
from pt_utils import  Embeddings, Trainer, VQVAE, data_sampler, Vqvae2Adaptive
from torch.utils import data
from torch import distributed as dist

from umap import UMAP
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from collections import Counter
from skimage import transform, metrics
import skimage
import pandas as pd
import numpy as np
import glob

from torchsummary import summary

from tqdm.notebook import trange, tqdm
import torchvision
from PIL import Image

import numpy as np
from sklearn.decomposition import PCA
from scipy.ndimage.filters import gaussian_filter
import scipy as sp
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import make_axes_locatable

import joblib
import pickle

seed = 51
np.random.seed(seed)
torch.manual_seed(seed)

# Create embeddings dataset

In [ ]:
# dataset_path = '../data/dataset_512/'
# dataset_path = '../datasets/bc_right_sub_left_minmax_4x_360'
# dataset_path = '../datasets/bc_left_4x_360'
# dataset_path = '../datasets/original/o_bc_left_9x_512_360'
dataset_path = '../datasets/original/o_bc_left_4x_768'

new_dataset_path='../datasets/original/o_bc_left_4x_768_embs'

resize_shape = (512, 512)
# resize_shape = (1024, 1024)

batch_size = 4

transform = transforms.Compose(
    [
        # transforms.Resize(resize_shape),
        # transforms.CenterCrop(resize_shape),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ]
)

vqvae2_model
dataset = datasets.ImageFolder(dataset_path, transform=transform)
device='cuda'

images_embs_t = []
images_embs_b = []

dataset_path = dataset.__dict__['root']
classes_folders = os.listdir(dataset_path)
classes_folders_images = [os.listdir(dataset_path + '/' + folder) for folder in classes_folders]
classes_folders_images_num = [len(os.listdir(dataset_path + '/' + folder)) for folder in classes_folders]

img_transform = dataset.__dict__['transform']

for i in range(len(classes_folders)):
    print(f'Number of folders {i + 1}/{len(classes_folders)}')
    for j in tqdm(range(classes_folders_images_num[i]), desc=f'Folder {classes_folders[i]}'):
        image_path = dataset_path + '/' + classes_folders[i] + '/' + classes_folders_images[i][j]

        image = Image.open(image_path)
        image = image.convert("RGB")
        image = img_transform(image)
        image = image.unsqueeze(0).to(device)
        image = transform(image)

        vqvae2_model.zero_grad()

        quant_t, quant_b, _, _, _ = vqvae2_model.encode(image)

        torch.save(quant_t,new_dataset_path+'/top_'+classes_folders_images[i][j])
        torch.save(quant_b,new_dataset_path+'/bottom_'+classes_folders_images[i][j])


In [ ]:
import argparse
import pickle

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import lmdb
from tqdm import tqdm

from dataset import ImageFileDataset, CodeRow
from vqvae import VQVAE


def extract(lmdb_env, loader, model, device):
    index = 0

    with lmdb_env.begin(write=True) as txn:
        pbar = tqdm(loader)

        for img, _, filename in pbar:
            img = img.to(device)

            _, _, _, id_t, id_b = model.encode(img)
            id_t = id_t.detach().cpu().numpy()
            id_b = id_b.detach().cpu().numpy()

            for file, top, bottom in zip(filename, id_t, id_b):
                row = CodeRow(top=top, bottom=bottom, filename=file)
                txn.put(str(index).encode('utf-8'), pickle.dumps(row))
                index += 1
                pbar.set_description(f'inserted: {index}')

        txn.put('length'.encode('utf-8'), str(index).encode('utf-8'))


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--size', type=int, default=256)
    parser.add_argument('--ckpt', type=str)
    parser.add_argument('--name', type=str)
    parser.add_argument('path', type=str)

    args = parser.parse_args()

    device = 'cuda'

    transform = transforms.Compose(
        [
            transforms.Resize(args.size),
            transforms.CenterCrop(args.size),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
        ]
    )

    dataset = ImageFileDataset(args.path, transform=transform)
    loader = DataLoader(dataset, batch_size=128, shuffle=False, num_workers=4)

    model = VQVAE()
    model.load_state_dict(torch.load(args.ckpt))
    model = model.to(device)
    model.eval()

    map_size = 100 * 1024 * 1024 * 1024

    env = lmdb.open(args.name, map_size=map_size)

    extract(env, loader, model, device)

# Train pixelsnail

In [7]:
class PixelTransform:
    def __init__(self):
        pass

    def __call__(self, input):
        ar = np.array(input)

        return torch.from_numpy(ar).long()

def train(args, epoch, loader, model, optimizer, scheduler, device):
    loader = tqdm(loader)

    criterion = nn.CrossEntropyLoss()

    for i, (top, bottom, label) in enumerate(loader):
        model.zero_grad()

        top = top.to(device)

        if args.hier == 'top':
            target = top
            out, _ = model(top)

        elif args.hier == 'bottom':
            bottom = bottom.to(device)
            target = bottom
            out, _ = model(bottom, condition=top)

        loss = criterion(out, target)
        loss.backward()

        if scheduler is not None:
            scheduler.step()
        optimizer.step()

        _, pred = out.max(1)
        correct = (pred == target).float()
        accuracy = correct.sum() / target.numel()

        lr = optimizer.param_groups[0]['lr']

        loader.set_description(
            (
                f'epoch: {epoch + 1}; loss: {loss.item():.5f}; '
                f'acc: {accuracy:.5f}; lr: {lr:.5f}'
            )
        )

In [ ]:

parser = argparse.ArgumentParser()
parser.add_argument('--batch', type=int, default=32)
parser.add_argument('--epoch', type=int, default=420)
parser.add_argument('--hier', type=str, default='top')
parser.add_argument('--lr', type=float, default=3e-4)
parser.add_argument('--channel', type=int, default=256)
parser.add_argument('--n_res_block', type=int, default=4)
parser.add_argument('--n_res_channel', type=int, default=256)
parser.add_argument('--n_out_res_block', type=int, default=0)
parser.add_argument('--n_cond_res_block', type=int, default=3)
parser.add_argument('--dropout', type=float, default=0.1)
parser.add_argument('--amp', type=str, default='O0')
parser.add_argument('--sched', type=str)
parser.add_argument('--ckpt', type=str)
parser.add_argument('path', type=str)


device = 'cuda'

dataset = LMDBDataset(args.path)
loader = DataLoader(
    dataset, batch_size=args.batch, shuffle=True, num_workers=4, drop_last=True
)

ckpt = {}

if args.ckpt is not None:
    ckpt = torch.load(args.ckpt)
    args = ckpt['args']

if args.hier == 'top':
    model = PixelSNAIL(
        [32, 32],
        512,
        args.channel,
        5,
        4,
        args.n_res_block,
        args.n_res_channel,
        dropout=args.dropout,
        n_out_res_block=args.n_out_res_block,
    )

elif args.hier == 'bottom':
    model = PixelSNAIL(
        [64, 64],
        512,
        args.channel,
        5,
        4,
        args.n_res_block,
        args.n_res_channel,
        attention=False,
        dropout=args.dropout,
        n_cond_res_block=args.n_cond_res_block,
        cond_res_channel=args.n_res_channel,
    )

if 'model' in ckpt:
    model.load_state_dict(ckpt['model'])

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=args.lr)

if amp is not None:
    model, optimizer = amp.initialize(model, optimizer, opt_level=args.amp)

model = nn.DataParallel(model)
model = model.to(device)

scheduler = None
if args.sched == 'cycle':
    scheduler = CycleScheduler(
        optimizer, args.lr, n_iter=len(loader) * args.epoch, momentum=None
    )

for i in range(args.epoch):
    train(args, i, loader, model, optimizer, scheduler, device)
    torch.save(
        {'model': model.module.state_dict(), 'args': args},
        f'checkpoint/pixelsnail_{args.hier}_{str(i + 1).zfill(3)}.pt',
    )